In [1]:
import tensorflow

tensorflow.config.experimental.list_physical_devices('GPU')

print("Num GPUs Available: ", len(tensorflow.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import  Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import datetime
import pickle
from numpy import array

pickle_in = open("x_train.pickle", "rb")
x_train = pickle.load(pickle_in)

pickle_in = open("x_test.pickle", "rb")
x_test = pickle.load(pickle_in)

pickle_in = open("y_train.pickle", "rb")
y_train = array(pickle.load(pickle_in))

pickle_in = open("y_test.pickle", "rb")
y_test = array(pickle.load(pickle_in))

# Normalize

x_train = x_train / 255.0
x_test = x_test / 255.0
#x_train = tensorflow.keras.utils.normalize(x_train, axis = 1) # try to use TS.normalize
#x_test = tensorflow.keras.utils.normalize(x_test, axis = 1)

In [ ]:
import datetime

dense_layers = [0] # [0, 1 2]
layers_sizes = [32] # [32, 64, 128, 256]
conv_layers = [3] # [1, 2, 3]

for dense_layer in dense_layers:
    for layers_size in layers_sizes:
        for conv_layer in conv_layers:
            NAME = "logs\\{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layers_size, dense_layer, datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
            
            model = Sequential()

            model.add( Conv2D(layers_size, (3,3), input_shape = x_train.shape[1:]) )
            model.add( Activation('relu') )
            model.add( MaxPooling2D(pool_size=(2,2)) )

            for layer in range(conv_layer - 1):
                model.add( Conv2D(layers_size, (3,3) ))
                model.add( Activation('relu') )
                model.add( MaxPooling2D(pool_size=(2,2)) )

            model.add( Flatten() )

            for layer in range(dense_layer):
                model.add( Dense(layers_size) )
                model.add( Activation('relu') )

            model.add( Dropout(0.2) )
            model.add( Dense(1) )
            model.add( Activation('sigmoid') )

            model.compile(loss='binary_crossentropy', #loss='categorical_crossentropy'
                        optimizer='adam', #optimizer='rmsprop
                        metrics=['accuracy'])

            tensorboard_callback = TensorBoard(log_dir=NAME, histogram_freq=1)
            
            print(NAME)

            model.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.2, callbacks=[tensorboard_callback])
            model.evaluate(x_test, y_test, verbose=2)

            model.save("3-conv-32-layer-size.model")